<a href="https://colab.research.google.com/github/isabellahyphen/mandelbrot/blob/main/mandelbrot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import numpy as np
import scipy.fftpack

# ffGn: fast (exact) fractional Gaussian noise and Brownian motion generator
# parameters: [N: Length of the output sequence; H: Hurst index; sigma: Standard deviation (optional, default=1); mu: Mean (optional, default=0)]
# returns: [y: generated noise sequence]

# frequency_values: values of frequencies for fft
# spectral_density: spectral density based on hurst index
# white_noise_frequency: white noise in frequency domain
# fgn_frequency: fractional gaussian noise in frequency domain
# time_domain_sequence: sequence in time domain after inverse fft
# noise_sequence`: final generated noise sequence

def ffGn(N, H, sigma=1, mu=0):
    # calculate the spectral density
    frequency_values = np.fft.fftfreq(N)
    frequency_values = np.hstack((frequency_values[1:N//2], frequency_values[-N//2+1:]))
    spectral_density = 1 / np.abs(frequency_values) ** (2 * H)

    # generate white noise
    white_noise_frequency = np.random.normal(0, 1, len(frequency_values)) + 1j * np.random.normal(0, 1, len(frequency_values))

    # generate the fGn
    fgn_frequency = np.sqrt(spectral_density / 2) * white_noise_frequency
    fgn_frequency = np.hstack((1, fgn_frequency, np.conj(fgn_frequency[::-1])))

    # inverse fft to obtain time domain sequence
    time_domain_sequence = np.fft.ifft(fgn_frequency).real
    time_domain_sequence = np.cumsum(time_domain_sequence)  # keep it if we want fBm, but comment out for fGn

    # rescale
    time_domain_sequence -= np.min(time_domain_sequence)
    time_domain_sequence /= np.max(time_domain_sequence)
    time_domain_sequence = time_domain_sequence * sigma + mu

    return time_domain_sequence

# test the function
ffGn(N=1000, H=0.5)[:10]

array([0.5817285 , 0.57989826, 0.57643034, 0.57455337, 0.57073899,
       0.56635588, 0.56023458, 0.55816057, 0.55722563, 0.55407164])

In [30]:
def lognormal_cascade(num_intervals, num_iterations, initial_value, lognorm_mean, lognorm_std_dev):

# it computes the random (lognormal) multifractal measure iteratively
# parameters: [num_intervals: number of intervals to be subdivided; num_iterations: number of iterations or stages;
# initial_value: initial value or area to be subdivided; lognorm_mean: mean parameter for the lognormal distribution;
# lognorm_std_dev: standard deviation parameter for the lognormal distribution;]
# returns: [weights: a row vector containing the weights after the last iteration; cdf_weights: cumulative distribution function of weights]

    weights = np.array([initial_value])

    for _ in range(num_iterations):
        lognorm_samples = np.random.lognormal(lognorm_mean, lognorm_std_dev, len(weights) * num_intervals)
        normalized_samples = lognorm_samples / np.sum(lognorm_samples)
        weights = np.repeat(weights, num_intervals) * normalized_samples

    cdf_weights = np.cumsum(weights)
    return weights, cdf_weights

# test it!
lognormal_cascade(num_intervals=2, num_iterations=3, initial_value=1, lognorm_mean=0, lognorm_std_dev=0.5)

(array([0.00331898, 0.00586959, 0.00577119, 0.0132711 , 0.02313874,
        0.01254962, 0.01517822, 0.03436535]),
 array([0.00331898, 0.00918857, 0.01495977, 0.02823086, 0.0513696 ,
        0.06391922, 0.07909745, 0.1134628 ]))

In [33]:
def mmar(num_subdivisions, num_iterations, Hurst_exponent, lognorm_mean, lognorm_std_dev):

# simulates a multifractal model of asset return using a multiplicative lognormal cascade
# parameters: [num_subdivisions: number of subdivisions (i.e. 2 for binomial model); num_iterations: number of iterations;
# the output sequence is of length num_subdivisions^num_iterations; hurst_exponent: for the fractional brownian motion;
# lognorm_mean: mean of the lognormal distribution; lognorm_std_dev: standard deviation for the lognormal distribution]
# returns: [asset_returns: simulated asset returns]

    sequence_length = num_subdivisions ** num_iterations
    lognorm_weights, _ = lognormal_cascade(num_subdivisions, num_iterations, 1, lognorm_mean, lognorm_std_dev)

    # adjust the length of the generated fGn to match lognorm_weights
    fGn_sequence = ffGn(len(lognorm_weights), Hurst_exponent)

    asset_returns = np.diff(fGn_sequence)[:len(lognorm_weights)] * lognorm_weights

    return asset_returns

# test our function
mmar(num_subdivisions=2, num_iterations=3, Hurst_exponent=0.5, lognorm_mean=0, lognorm_std_dev=0.5)[:10]


array([ 6.70899637e-04,  1.48700516e-04, -4.90295278e-04, -9.32312776e-05,
       -8.88085074e-04, -4.74356259e-03, -9.22961796e-03,  1.27962456e-02])